In [1]:
#import libraries
import pandas as pd
import numpy as np
from numpy import asarray
import os
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
import scipy
import folium
import json
import requests
import datetime
import sklearn
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OrdinalEncoder

In [2]:
path = r'/Users/golde/Crime Analysis/'

In [3]:
crimes = pd.read_csv(os.path.join(path, '2. Data', 'Original Data', 'crimes1.csv'))

In [4]:
pd.options.display.max_columns = None

In [5]:
crimes.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards,Boundaries - ZIP Codes,Police Districts,Police Beats
0,10224738,HY411648,09/05/2015 01:30:00 PM,043XX S WOOD ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,924,9.0,12.0,61.0,08B,1165074.0,1875917.0,2015,02/10/2018 03:50:01 PM,41.815117,-87.670000,"(41.815117282, -87.669999562)",29.0,14924.0,59.0,706.0,3.0,37.0,23.0,108.0
1,10224739,HY411615,09/04/2015 11:30:00 AM,008XX N CENTRAL AVE,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,1511,15.0,29.0,25.0,06,1138875.0,1904869.0,2015,02/10/2018 03:50:01 PM,41.895080,-87.765400,"(41.895080471, -87.765400451)",4.0,4299.0,26.0,562.0,45.0,5.0,25.0,67.0
2,11646166,JC213529,09/01/2018 12:01:00 AM,082XX S INGLESIDE AVE,0810,THEFT,OVER $500,RESIDENCE,False,True,631,6.0,8.0,44.0,06,NaN,NaN,2018,04/06/2019 04:04:43 PM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10224740,HY411595,09/05/2015 12:45:00 PM,035XX W BARRY AVE,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,1412,14.0,35.0,21.0,18,1152037.0,1920384.0,2015,02/10/2018 03:50:01 PM,41.937406,-87.716650,"(41.937405765, -87.716649687)",15.0,21538.0,22.0,216.0,12.0,39.0,7.0,168.0
4,10224741,HY411610,09/05/2015 01:00:00 PM,0000X N LARAMIE AVE,0560,ASSAULT,SIMPLE,APARTMENT,False,True,1522,15.0,28.0,25.0,08A,1141706.0,1900086.0,2015,02/10/2018 03:50:01 PM,41.881903,-87.755121,"(41.881903443, -87.755121152)",11.0,22216.0,26.0,696.0,23.0,32.0,25.0,81.0


In [6]:
crimes = crimes.drop(columns = ['Historical Wards 2003-2015', 'Zip Codes', 'Community Areas', 'Boundaries - ZIP Codes', 'Police Districts', 'Police Beats'])

In [7]:
crimes = crimes.dropna(how = 'any')

In [8]:
crimes.isnull().sum()

ID                      0
Case Number             0
Date                    0
Block                   0
IUCR                    0
Primary Type            0
Description             0
Location Description    0
Arrest                  0
Domestic                0
Beat                    0
District                0
Ward                    0
Community Area          0
FBI Code                0
X Coordinate            0
Y Coordinate            0
Year                    0
Updated On              0
Latitude                0
Longitude               0
Location                0
Census Tracts           0
Wards                   0
dtype: int64

In [9]:
# duplicates
crimes_dups = crimes[crimes.duplicated()]

In [10]:
# checking for mixed types
for col in crimes.columns.tolist():
  weird = (crimes[[col]].applymap(type) != crimes[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (crimes[weird]) > 0:
    print (col)

In [11]:
crimes.loc[(crimes['Community Area'] == 1), 'Neighborhood'] = 'Rogers Park'
crimes.loc[(crimes['Community Area'] == 2), 'Neighborhood'] = 'West Ridge'
crimes.loc[(crimes['Community Area'] == 3), 'Neighborhood'] = 'Uptown'
crimes.loc[(crimes['Community Area'] == 4), 'Neighborhood'] = 'Lincoln Square'
crimes.loc[(crimes['Community Area'] == 5), 'Neighborhood'] = 'North Center'
crimes.loc[(crimes['Community Area'] == 6), 'Neighborhood'] = 'Lake View'
crimes.loc[(crimes['Community Area'] == 7), 'Neighborhood'] = 'Lincoln Park'
crimes.loc[(crimes['Community Area'] == 8), 'Neighborhood'] = 'Near North Side'
crimes.loc[(crimes['Community Area'] == 9), 'Neighborhood'] = 'Edison Park'
crimes.loc[(crimes['Community Area'] == 10), 'Neighborhood'] = 'Norwood Park'
crimes.loc[(crimes['Community Area'] == 11), 'Neighborhood'] = 'Jefferson Park'
crimes.loc[(crimes['Community Area'] == 12), 'Neighborhood'] = 'Forest Glen'
crimes.loc[(crimes['Community Area'] == 13), 'Neighborhood'] = 'North Park'
crimes.loc[(crimes['Community Area'] == 14), 'Neighborhood'] = 'Albany Park'
crimes.loc[(crimes['Community Area'] == 15), 'Neighborhood'] = 'Portage Park'
crimes.loc[(crimes['Community Area'] == 16), 'Neighborhood'] = 'Irving Park'
crimes.loc[(crimes['Community Area'] == 17), 'Neighborhood'] = 'Dunning'
crimes.loc[(crimes['Community Area'] == 18), 'Neighborhood'] = 'Montclare'
crimes.loc[(crimes['Community Area'] == 19), 'Neighborhood'] = 'Belmont Cragin'
crimes.loc[(crimes['Community Area'] == 20), 'Neighborhood'] = 'Hermosa'
crimes.loc[(crimes['Community Area'] == 21), 'Neighborhood'] = 'Avondale'
crimes.loc[(crimes['Community Area'] == 22), 'Neighborhood'] = 'Logan Square'
crimes.loc[(crimes['Community Area'] == 23), 'Neighborhood'] = 'Humboldt Park'
crimes.loc[(crimes['Community Area'] == 24), 'Neighborhood'] = 'West Town'
crimes.loc[(crimes['Community Area'] == 25), 'Neighborhood'] = 'Austin'
crimes.loc[(crimes['Community Area'] == 26), 'Neighborhood'] = 'West Garfield Park'
crimes.loc[(crimes['Community Area'] == 27), 'Neighborhood'] = 'East Garfield Park'
crimes.loc[(crimes['Community Area'] == 28), 'Neighborhood'] = 'Near West Side'
crimes.loc[(crimes['Community Area'] == 29), 'Neighborhood'] = 'North Lawndale'
crimes.loc[(crimes['Community Area'] == 30), 'Neighborhood'] = 'South Lawndale'
crimes.loc[(crimes['Community Area'] == 31), 'Neighborhood'] = 'Lower West Side'
crimes.loc[(crimes['Community Area'] == 32), 'Neighborhood'] = '(The) Loop[11]'
crimes.loc[(crimes['Community Area'] == 33), 'Neighborhood'] = 'Near South Side'
crimes.loc[(crimes['Community Area'] == 34), 'Neighborhood'] = 'Armour Square'
crimes.loc[(crimes['Community Area'] == 35), 'Neighborhood'] = 'Douglas'
crimes.loc[(crimes['Community Area'] == 36), 'Neighborhood'] = 'Oakland'
crimes.loc[(crimes['Community Area'] == 37), 'Neighborhood'] = 'Fuller Park'
crimes.loc[(crimes['Community Area'] == 38), 'Neighborhood'] = 'Grand Boulevard'
crimes.loc[(crimes['Community Area'] == 39), 'Neighborhood'] = 'Kenwood'
crimes.loc[(crimes['Community Area'] == 40), 'Neighborhood'] = 'Washington Park'
crimes.loc[(crimes['Community Area'] == 41), 'Neighborhood'] = 'Hyde Park'
crimes.loc[(crimes['Community Area'] == 42), 'Neighborhood'] = 'Woodlawn'
crimes.loc[(crimes['Community Area'] == 43), 'Neighborhood'] = 'South Shore'
crimes.loc[(crimes['Community Area'] == 44), 'Neighborhood'] = 'Chatham'
crimes.loc[(crimes['Community Area'] == 45), 'Neighborhood'] = 'Avalon Park'
crimes.loc[(crimes['Community Area'] == 46), 'Neighborhood'] = 'South Chicago'
crimes.loc[(crimes['Community Area'] == 47), 'Neighborhood'] = 'Burnside'
crimes.loc[(crimes['Community Area'] == 48), 'Neighborhood'] = 'Calumet Heights'
crimes.loc[(crimes['Community Area'] == 49), 'Neighborhood'] = 'Roseland'
crimes.loc[(crimes['Community Area'] == 50), 'Neighborhood'] = 'Pullman'
crimes.loc[(crimes['Community Area'] == 51), 'Neighborhood'] = 'South Deering'
crimes.loc[(crimes['Community Area'] == 52), 'Neighborhood'] = 'East Side'
crimes.loc[(crimes['Community Area'] == 53), 'Neighborhood'] = 'West Pullman'
crimes.loc[(crimes['Community Area'] == 54), 'Neighborhood'] = 'Riverdale'
crimes.loc[(crimes['Community Area'] == 55), 'Neighborhood'] = 'Hegewisch'
crimes.loc[(crimes['Community Area'] == 56), 'Neighborhood'] = 'Garfield Ridge'
crimes.loc[(crimes['Community Area'] == 57), 'Neighborhood'] = 'Archer Heights'
crimes.loc[(crimes['Community Area'] == 58), 'Neighborhood'] = 'Brighton Park'
crimes.loc[(crimes['Community Area'] == 59), 'Neighborhood'] = 'McKinley Park'
crimes.loc[(crimes['Community Area'] == 60), 'Neighborhood'] = 'Bridgeport'
crimes.loc[(crimes['Community Area'] == 61), 'Neighborhood'] = 'New City'
crimes.loc[(crimes['Community Area'] == 62), 'Neighborhood'] = 'West Elsdon'
crimes.loc[(crimes['Community Area'] == 63), 'Neighborhood'] = 'Gage Park'
crimes.loc[(crimes['Community Area'] == 64), 'Neighborhood'] = 'Clearing'
crimes.loc[(crimes['Community Area'] == 65), 'Neighborhood'] = 'West Lawn'
crimes.loc[(crimes['Community Area'] == 66), 'Neighborhood'] = 'Chicago Lawn'
crimes.loc[(crimes['Community Area'] == 67), 'Neighborhood'] = 'West Englewood'
crimes.loc[(crimes['Community Area'] == 68), 'Neighborhood'] = 'Englewood'
crimes.loc[(crimes['Community Area'] == 69), 'Neighborhood'] = 'Greater Grand Crossing'
crimes.loc[(crimes['Community Area'] == 70), 'Neighborhood'] = 'Ashburn'
crimes.loc[(crimes['Community Area'] == 71), 'Neighborhood'] = 'Auburn Gresham'
crimes.loc[(crimes['Community Area'] == 72), 'Neighborhood'] = 'Beverly'
crimes.loc[(crimes['Community Area'] == 73), 'Neighborhood'] = 'Washington Heights'
crimes.loc[(crimes['Community Area'] == 74), 'Neighborhood'] = 'Mount Greenwood'
crimes.loc[(crimes['Community Area'] == 75), 'Neighborhood'] = 'Morgan Park'
crimes.loc[(crimes['Community Area'] == 76), 'Neighborhood'] = 'Ohare'
crimes.loc[(crimes['Community Area'] == 77), 'Neighborhood'] = 'Edgewater'

In [12]:
crimes.loc[(crimes['Community Area'] == 1), 'Rogers Park'] = '1.0'
crimes.loc[(crimes['Community Area'] > 1), 'Rogers Park'] = '0.0'
crimes.loc[(crimes['Community Area'] == 2), 'West Ridge'] = '1.0'
crimes.loc[(crimes['Community Area'] > 2) | (crimes['Community Area'] < 2), 'West Ridge'] = '0.0'
crimes.loc[(crimes['Community Area'] == 3), 'Uptown'] = '1.0'
crimes.loc[(crimes['Community Area'] > 3) | (crimes['Community Area'] > 3), 'Uptown'] = '0.0'
crimes.loc[(crimes['Community Area'] == 4), 'Lincoln Square'] = '1.0'
crimes.loc[(crimes['Community Area'] > 4) | (crimes['Community Area'] < 4), 'Lincoln Square'] = '0.0'
crimes.loc[(crimes['Community Area'] == 5), 'North Center'] = '1.0'
crimes.loc[(crimes['Community Area'] > 5) | (crimes['Community Area'] < 5), 'North Center'] = '0.0'
crimes.loc[(crimes['Community Area'] == 6), 'Lake View'] = '1.0'
crimes.loc[(crimes['Community Area'] > 6) | (crimes['Community Area'] < 6), 'Lake View'] = '0.0'
crimes.loc[(crimes['Community Area'] == 7), 'Lincoln Park'] = '1.0'
crimes.loc[(crimes['Community Area'] > 7) | (crimes['Community Area'] < 7), 'Lincoln Park'] = '0.0'
crimes.loc[(crimes['Community Area'] == 8), 'Near North Side'] = '1.0'
crimes.loc[(crimes['Community Area'] > 8) | (crimes['Community Area'] < 8), 'Near North Side'] = '0.0'
crimes.loc[(crimes['Community Area'] == 9), 'Edison Park'] = '1.0'
crimes.loc[(crimes['Community Area'] > 9) | (crimes['Community Area'] < 9), 'Edison Park'] = '0.0'
crimes.loc[(crimes['Community Area'] == 10), 'Norwood Park'] = '1.0'
crimes.loc[(crimes['Community Area'] > 10) | (crimes['Community Area'] < 10), 'Norwood Park'] = '0.0'
crimes.loc[(crimes['Community Area'] == 11), 'Jefferson Park'] = '1.0'
crimes.loc[(crimes['Community Area'] > 11) | (crimes['Community Area'] < 11), 'Jefferson Park'] = '0.0'
crimes.loc[(crimes['Community Area'] == 12), 'Forest Glen'] = '1.0'
crimes.loc[(crimes['Community Area'] > 12) | (crimes['Community Area'] < 12), 'Forest Glen'] = '0.0'
crimes.loc[(crimes['Community Area'] == 13), 'North Park'] = '1.0'
crimes.loc[(crimes['Community Area'] > 13) | (crimes['Community Area'] < 13), 'North Park'] = '0.0'
crimes.loc[(crimes['Community Area'] == 14), 'Albany Park'] = '1.0'
crimes.loc[(crimes['Community Area'] > 14) | (crimes['Community Area'] < 14), 'Albany Park'] = '0.0'
crimes.loc[(crimes['Community Area'] == 15), 'Portage Park'] = '1.0'
crimes.loc[(crimes['Community Area'] > 15) | (crimes['Community Area'] < 15), 'Portage Park'] = '0.0'
crimes.loc[(crimes['Community Area'] == 16), 'Irving Park'] = '1.0'
crimes.loc[(crimes['Community Area'] > 16) | (crimes['Community Area'] < 16), 'Irving Park'] = '0.0'
crimes.loc[(crimes['Community Area'] == 17), 'Dunning'] = '1.0'
crimes.loc[(crimes['Community Area'] > 17) | (crimes['Community Area'] < 17), 'Dunning'] = '0.0'
crimes.loc[(crimes['Community Area'] == 18), 'Montclare'] = '1.0'
crimes.loc[(crimes['Community Area'] > 18) | (crimes['Community Area'] < 18), 'Montclare'] = '0.0'
crimes.loc[(crimes['Community Area'] == 19), 'Belmont Cragin'] = '1.0'
crimes.loc[(crimes['Community Area'] > 19) | (crimes['Community Area'] < 19), 'Belmont Cragin'] = '0.0'
crimes.loc[(crimes['Community Area'] == 20), 'Hermosa'] = '1.0'
crimes.loc[(crimes['Community Area'] > 20) | (crimes['Community Area'] < 20), 'Hermosa'] = '0.0'
crimes.loc[(crimes['Community Area'] == 21), 'Avondale'] = '1.0'
crimes.loc[(crimes['Community Area'] > 21) | (crimes['Community Area'] < 21), 'Avondale'] = '0.0'
crimes.loc[(crimes['Community Area'] == 22), 'Logan Square'] = '1.0'
crimes.loc[(crimes['Community Area'] > 22) | (crimes['Community Area'] < 22), 'Logan Square'] = '0.0'
crimes.loc[(crimes['Community Area'] == 23), 'Humboldt Park'] = '1.0'
crimes.loc[(crimes['Community Area'] > 23) | (crimes['Community Area'] < 23), 'Humboldt Park'] = '0.0'
crimes.loc[(crimes['Community Area'] == 24), 'West Town'] = '1.0'
crimes.loc[(crimes['Community Area'] > 24) | (crimes['Community Area'] < 24), 'West Town'] = '0.0'
crimes.loc[(crimes['Community Area'] == 25), 'Austin'] = '1.0'
crimes.loc[(crimes['Community Area'] > 25) | (crimes['Community Area'] < 25), 'Austin'] = '0.0'
crimes.loc[(crimes['Community Area'] == 26), 'West Garfield Park'] = '1.0'
crimes.loc[(crimes['Community Area'] > 26) | (crimes['Community Area'] < 26), 'West Garfield Park'] = '0.0'
crimes.loc[(crimes['Community Area'] == 27), 'East Garfield Park'] = '1.0'
crimes.loc[(crimes['Community Area'] > 27) | (crimes['Community Area'] < 27), 'East Garfield Park'] = '0.0'
crimes.loc[(crimes['Community Area'] == 28), 'Near West Side'] = '1.0'
crimes.loc[(crimes['Community Area'] > 28) | (crimes['Community Area'] < 28), 'Near West Side'] = '0.0'
crimes.loc[(crimes['Community Area'] == 29), 'North Lawndale'] = '1.0'
crimes.loc[(crimes['Community Area'] > 29) | (crimes['Community Area'] < 29), 'North Lawndale'] = '0.0'
crimes.loc[(crimes['Community Area'] == 30), 'South Lawndale'] = '1.0'
crimes.loc[(crimes['Community Area'] > 30) | (crimes['Community Area'] < 30), 'South Lawndale'] = '0.0'
crimes.loc[(crimes['Community Area'] == 31), 'Lower West Side'] = '1.0'
crimes.loc[(crimes['Community Area'] > 31) | (crimes['Community Area'] < 31), 'Lower West Side'] = '0.0'
crimes.loc[(crimes['Community Area'] == 32), '(The) Loop[11]'] = '1.0'
crimes.loc[(crimes['Community Area'] > 32) | (crimes['Community Area'] < 32), '(The) Loop[11]'] = '0.0'
crimes.loc[(crimes['Community Area'] == 33), 'Near South Side'] = '1.0'
crimes.loc[(crimes['Community Area'] > 33) | (crimes['Community Area'] < 33), 'Near South Side'] = '0.0'
crimes.loc[(crimes['Community Area'] == 34), 'Armour Square'] = '1.0'
crimes.loc[(crimes['Community Area'] > 34) | (crimes['Community Area'] < 34), 'Armour Square'] = '0.0'
crimes.loc[(crimes['Community Area'] == 35), 'Douglas'] = '1.0'
crimes.loc[(crimes['Community Area'] > 35) | (crimes['Community Area'] < 35), 'Douglas'] = '0.0'
crimes.loc[(crimes['Community Area'] == 36), 'Oakland'] = '1.0'
crimes.loc[(crimes['Community Area'] > 36) | (crimes['Community Area'] < 36), 'Oakland'] = '0.0'
crimes.loc[(crimes['Community Area'] == 37), 'Fuller Park'] = '1.0'
crimes.loc[(crimes['Community Area'] > 37) | (crimes['Community Area'] < 37), 'Fuller Park'] = '0.0'
crimes.loc[(crimes['Community Area'] == 38), 'Grand Boulevard'] = '1.0'
crimes.loc[(crimes['Community Area'] > 38) | (crimes['Community Area'] < 38), 'Grand Boulevard'] = '0.0'
crimes.loc[(crimes['Community Area'] == 39), 'Kenwood'] = '1.0'
crimes.loc[(crimes['Community Area'] > 39) | (crimes['Community Area'] < 39), 'Kenwood'] = '0.0'
crimes.loc[(crimes['Community Area'] == 40), 'Washington Park'] = '1.0'
crimes.loc[(crimes['Community Area'] > 40) | (crimes['Community Area'] < 40), 'Washington Park'] = '0.0'
crimes.loc[(crimes['Community Area'] == 41), 'Hyde Park'] = '1.0'
crimes.loc[(crimes['Community Area'] > 41) | (crimes['Community Area'] < 41), 'Hyde Park'] = '0.0'
crimes.loc[(crimes['Community Area'] == 42), 'Woodlawn'] = '1.0'
crimes.loc[(crimes['Community Area'] > 42) | (crimes['Community Area'] < 42), 'Woodlawn'] = '0.0'
crimes.loc[(crimes['Community Area'] == 43), 'South Shore'] = '1.0'
crimes.loc[(crimes['Community Area'] > 43) | (crimes['Community Area'] < 43), 'South Shore'] = '0.0'
crimes.loc[(crimes['Community Area'] == 44), 'Chatham'] = '1.0'
crimes.loc[(crimes['Community Area'] > 44) | (crimes['Community Area'] < 44), 'Chatham'] = '0.0'
crimes.loc[(crimes['Community Area'] == 45), 'Avalon Park'] = '1.0'
crimes.loc[(crimes['Community Area'] > 45) | (crimes['Community Area'] < 45), 'Avalon Park'] = '0.0'
crimes.loc[(crimes['Community Area'] == 46), 'South Chicago'] = '1.0'
crimes.loc[(crimes['Community Area'] > 46) | (crimes['Community Area'] < 46), 'South Chicago'] = '0.0'
crimes.loc[(crimes['Community Area'] == 47), 'Burnside'] = '1.0'
crimes.loc[(crimes['Community Area'] > 47) | (crimes['Community Area'] < 47), 'Burnside'] = '0.0'
crimes.loc[(crimes['Community Area'] == 48), 'Calumet Heights'] = '1.0'
crimes.loc[(crimes['Community Area'] > 48) | (crimes['Community Area'] < 48), 'Calumet Heights'] = '0.0'
crimes.loc[(crimes['Community Area'] == 49), 'Roseland'] = '1.0'
crimes.loc[(crimes['Community Area'] > 49) | (crimes['Community Area'] < 49), 'Roseland'] = '0.0'
crimes.loc[(crimes['Community Area'] == 50), 'Pullman'] = '1.0'
crimes.loc[(crimes['Community Area'] > 50) | (crimes['Community Area'] < 50), 'Pullman'] = '0.0'
crimes.loc[(crimes['Community Area'] == 51), 'South Deering'] = '1.0'
crimes.loc[(crimes['Community Area'] > 51) | (crimes['Community Area'] < 51), 'South Deering'] = '0.0'
crimes.loc[(crimes['Community Area'] == 52), 'East Side'] = '1.0'
crimes.loc[(crimes['Community Area'] > 52) | (crimes['Community Area'] < 52), 'East Side'] = '0.0'
crimes.loc[(crimes['Community Area'] == 53), 'West Pullman'] = '1.0'
crimes.loc[(crimes['Community Area'] > 53) | (crimes['Community Area'] < 53), 'West Pullman'] = '0.0'
crimes.loc[(crimes['Community Area'] == 54), 'Riverdale'] = '1.0'
crimes.loc[(crimes['Community Area'] > 54) | (crimes['Community Area'] < 54), 'Riverdale'] = '0.0'
crimes.loc[(crimes['Community Area'] == 55), 'Hegewisch'] = '1.0'
crimes.loc[(crimes['Community Area'] > 55) | (crimes['Community Area'] < 55), 'Hegewisch'] = '0.0'
crimes.loc[(crimes['Community Area'] == 56), 'Garfield Ridge'] = '1.0'
crimes.loc[(crimes['Community Area'] > 56) | (crimes['Community Area'] < 56), 'Garfield Ridge'] = '0.0'
crimes.loc[(crimes['Community Area'] == 57), 'Archer Heights'] = '1.0'
crimes.loc[(crimes['Community Area'] > 57) | (crimes['Community Area'] < 57), 'Archer Heights'] = '0.0'
crimes.loc[(crimes['Community Area'] == 58), 'Brighton Park'] = '1.0'
crimes.loc[(crimes['Community Area'] > 58) | (crimes['Community Area'] < 58), 'Brighton Park'] = '0.0'
crimes.loc[(crimes['Community Area'] == 59), 'McKinley Park'] = '1.0'
crimes.loc[(crimes['Community Area'] > 59) | (crimes['Community Area'] < 59), 'McKinley Park'] = '0.0'
crimes.loc[(crimes['Community Area'] == 60), 'Bridgeport'] = '1.0'
crimes.loc[(crimes['Community Area'] > 60) | (crimes['Community Area'] < 60), 'Bridgeport'] = '0.0'
crimes.loc[(crimes['Community Area'] == 61), 'New City'] = '1.0'
crimes.loc[(crimes['Community Area'] > 61) | (crimes['Community Area'] < 61), 'New City'] = '0.0'
crimes.loc[(crimes['Community Area'] == 62), 'West Elsdon'] = '1.0'
crimes.loc[(crimes['Community Area'] > 62) | (crimes['Community Area'] < 62), 'West Elsdon'] = '0.0'
crimes.loc[(crimes['Community Area'] == 63), 'Gage Park'] = '1.0'
crimes.loc[(crimes['Community Area'] > 63) | (crimes['Community Area'] < 63), 'Gage Park'] = '0.0'
crimes.loc[(crimes['Community Area'] == 64), 'Clearing'] = '1.0'
crimes.loc[(crimes['Community Area'] > 64) | (crimes['Community Area'] < 64), 'Clearing'] = '0.0'
crimes.loc[(crimes['Community Area'] == 65), 'West Lawn'] = '1.0'
crimes.loc[(crimes['Community Area'] > 65) | (crimes['Community Area'] < 65), 'West Lawn'] = '0.0'
crimes.loc[(crimes['Community Area'] == 66), 'Chicago Lawn'] = '1.0'
crimes.loc[(crimes['Community Area'] > 66) | (crimes['Community Area'] < 66), 'Chicago Lawn'] = '0.0'
crimes.loc[(crimes['Community Area'] == 67), 'West Englewood'] = '1.0'
crimes.loc[(crimes['Community Area'] > 67) | (crimes['Community Area'] < 67), 'West Englewood'] = '0.0'
crimes.loc[(crimes['Community Area'] == 68), 'Englewood'] = '1.0'
crimes.loc[(crimes['Community Area'] > 68) | (crimes['Community Area'] < 68), 'Englewood'] = '0.0'
crimes.loc[(crimes['Community Area'] == 69), 'Greater Grand Crossing'] = '1.0'
crimes.loc[(crimes['Community Area'] > 69) | (crimes['Community Area'] < 69), 'Greater Grand Crossing'] = '0.0'
crimes.loc[(crimes['Community Area'] == 70), 'Ashburn'] = '1.0'
crimes.loc[(crimes['Community Area'] > 70) | (crimes['Community Area'] < 70), 'Ashburn'] = '0.0'
crimes.loc[(crimes['Community Area'] == 71), 'Auburn Gresham'] = '1.0'
crimes.loc[(crimes['Community Area'] > 71) | (crimes['Community Area'] < 71), 'Auburn Gresham'] = '0.0'
crimes.loc[(crimes['Community Area'] == 72), 'Beverly'] = '1.0'
crimes.loc[(crimes['Community Area'] > 72) | (crimes['Community Area'] < 72), 'Beverly'] = '0.0'
crimes.loc[(crimes['Community Area'] == 73), 'Washington Heights'] = '1.0'
crimes.loc[(crimes['Community Area'] > 73) | (crimes['Community Area'] < 73), 'Washington Heights'] = '0.0'
crimes.loc[(crimes['Community Area'] == 74), 'Mount Greenwood'] = '1.0'
crimes.loc[(crimes['Community Area'] > 74) | (crimes['Community Area'] < 74), 'Mount Greenwood'] = '0.0'
crimes.loc[(crimes['Community Area'] == 75), 'Morgan Park'] = '1.0'
crimes.loc[(crimes['Community Area'] > 75) | (crimes['Community Area'] < 75), 'Morgan Park'] = '0.0'
crimes.loc[(crimes['Community Area'] == 76), 'Ohare'] = '1.0'
crimes.loc[(crimes['Community Area'] > 76) | (crimes['Community Area'] < 76), 'Ohare'] = '0.0'
crimes.loc[(crimes['Community Area'] == 77), 'Edgewater'] = '1.0'
crimes.loc[(crimes['Community Area'] > 77) | (crimes['Community Area'] < 77), 'Edgewater'] = '0.0'

In [13]:
#Create Crimes
crimes.loc[(crimes['Primary Type'] == 'BATTERY') | (crimes['Primary Type'] == 'ROBBERY') | (crimes['Primary Type'] == 'WEAPONS VIOLATION') | (crimes['Primary Type'] == 'OFFENSE INVOLVING CHILDREN') | (crimes['Primary Type'] == 'SEX OFFENSE') | (crimes['Primary Type'] == 'CRIM SEXUAL ASSAULT') | (crimes['Primary Type'] == 'ARSON') | (crimes['Primary Type'] == 'KIDNAPPING') | (crimes['Primary Type'] == 'CRIMINAL SEXUAL ASSAULT') | (crimes['Primary Type'] == 'HOMICIDE') | (crimes['Primary Type'] == 'HUMAN TRAFFICKING'), 'Crime Type'] = 'Violent'
crimes.loc[(crimes['Primary Type'] == 'THEFT') | (crimes['Primary Type'] == 'NARCOTICS') | (crimes['Primary Type'] == 'ASSAULT') | (crimes['Primary Type'] == 'BURGLARY') | (crimes['Primary Type'] ==  'OTHER OFFENSE') | (crimes['Primary Type'] == 'CRIMINAL DAMAGE') | (crimes['Primary Type'] ==  'DECEPTIVE PRACTICE') | (crimes['Primary Type'] == 'CRIMINAL TRESPASS') | (crimes['Primary Type'] == 'MOTOR VEHICLE THEFT') | (crimes['Primary Type'] == 'SEX OFFENSE') | (crimes['Primary Type'] == 'INTERFERENCE WITH PUBLIC OFFICER') | (crimes['Primary Type'] == 'PUBLIC PEACE VIOLATION') | (crimes['Primary Type'] == 'PROSTITUTION') | (crimes['Primary Type'] == 'GAMBLING') | (crimes['Primary Type'] == 'LIQUOR LAW VIOLATION') | (crimes['Primary Type'] == 'STALKING') | (crimes['Primary Type'] == 'INTIMIDATION') | (crimes['Primary Type'] == 'CONCEALED CARRY LICENSE VIOLATION') | (crimes['Primary Type'] == 'NON - CRIMINAL') | (crimes['Primary Type'] == 'OBSCENITY') | (crimes['Primary Type'] == 'PUBLIC INDECENCY') | (crimes['Primary Type'] == 'OTHER NARCOTIC VIOLATION') | (crimes['Primary Type'] == 'NON-CRIMINAL') | (crimes['Primary Type'] == 'NON-CRIMINAL (SUBJECT SPECIFIED)'), 'Crime Type'] = 'Non-Violent'

In [14]:
crimes["Date"] = pd.to_datetime(crimes["Date"])

In [15]:
crimes['New_Dates'] = pd.to_datetime(crimes['Date']).dt.date

In [16]:
crimes['Time'] = pd.to_datetime(crimes['Date']).dt.time

In [17]:
crimes["New_Dates"] = pd.to_datetime(crimes["New_Dates"])

In [18]:
crimes['Day'] = crimes['New_Dates'].dt.day
crimes['Month'] = crimes['New_Dates'].dt.month
crimes['Year'] = crimes['New_Dates'].dt.year

In [19]:
#Create Seasons
crimes.loc[(crimes['Month'] == 3) | (crimes['Month'] == 4) | (crimes['Month'] == 5), 'Season'] = 'Spring'
crimes.loc[(crimes['Month'] == 6) | (crimes['Month'] == 7) | (crimes['Month'] == 8), 'Season'] = 'Summer'
crimes.loc[(crimes['Month'] == 9) | (crimes['Month'] == 10) | (crimes['Month'] == 11), 'Season'] = 'Fall'
crimes.loc[(crimes['Month'] == 12) | (crimes['Month'] == 1) | (crimes['Month'] == 2), 'Season'] = 'Winter'

In [20]:
#export data
crimes.to_csv(os.path.join(path, '2. Data','Prepared Data', 'crimes_cleaned_1.csv'))